In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def ll_to_xyz(lat,lon):
    R=6371 #in KM
    lat=lat*np.pi/180.0
    lon=lon*np.pi/180.0
    x=R*np.cos(lat)*np.cos(lon)
    y=R*np.cos(lat)*np.sin(lon)
    z=R*np.sin(lat)
    return x,y,z
def xyz_to_ll(l):
    R=6371 # in KM
    lat=np.arcsin(l[2]/R)*180.0/np.pi
    lon=np.arctan2(l[1],l[0])*180.0/np.pi
    return lat,lon
def centroid(arr):
    l = len(arr)
    s=np.sum(arr,axis=0)
    s/=l
    return s 

In [3]:
u_rev=pd.read_csv('total_user_review.csv',index_col='User_id',low_memory=False)

In [ ]:
u_rev=u_rev.drop(columns=['Review_id','User_name','Rating','Datetime'])

In [ ]:
ids=list(set(u_rev.index))

In [ ]:
f={}
q=0
for i in range(len(ids)):
    k=0
    t1=pd.DataFrame(u_rev.loc[ids[i]])
    res=[]
    for index,row in t1.iterrows():
        try:
            review=str(row['Review']).lower()
            if 'deliver' in review or 'swiggy' in review or 'foodpanda' in review or 'packing' in review or 'sent' in review:
                res.append(str(row['Name']).strip())
                k+=1
        except:
            pass
    if k>0:
        q+=1
        print('\t{}'.format(q),end='\r')
        f[i]=list(set(res))

In [ ]:
wr=['user_id,res_list']
for i in f:
    l=','.join(f[i])
    wr.append('{},"{}"'.format(ids[i],l))
#wr='\n'.format(wr)

wr='\n'.join(wr)

with open('user_del_res.csv','w',encoding='utf8') as fl:
    fl.write(wr)



In [ ]:
req=[ids[i] for i in f.keys()]

In [ ]:
fi={}
q=0
for r in req:
    k=0
    t1=pd.DataFrame(u_rev.loc[r])
    try:
        res=[x.strip() for x in t1.Name]
    except:
        res=[]
    if len(res)>0:
        q+=1
        print('\t{}'.format(q),end='\r')
        fi[r]=list(set(res))

In [ ]:
wr=['user_id,res_list']
for i in fi:
    l=','.join(fi[i])
    wr.append('{},"{}"'.format(i,l))
#wr='\n'.format(wr)

wr='\n'.join(wr)

with open('user_res_list.csv','w',encoding='utf8') as fl:
    fl.write(wr)



In [ ]:
def res_to_xyz(res):
    l=[]
    for r in res:
        try:
            f={}
            f['x'],f['y'],f['z']=ll_to_xyz(lcd.loc[r]['lat'],lcd.loc[r]['long'])
            l.append(f)
        except Exception as e:
            #print(e)
            pass
    return pd.DataFrame(l)
def df_to_labels(df):
    db=DBSCAN(8,3)
    db.fit_predict(df)
    return db.labels_
def final_xyz(df,labels):
    l=[]
    f={}
    for i in range(df.shape[0]):
        if labels[i]==-1:
            l.append(list(df.iloc[i]))
        else:
            lb=labels[i]
            if lb not in f:
                f[lb]=[]
            f[lb].append(list(df.iloc[i]))
    for lb in f:
        l.append(centroid(f[lb]))
    return l
def get_centroid_ll(df,labels):
    ll=final_xyz(df,labels)
    for i in range(len(ll)):
        ll[i]=xyz_to_ll(ll[i])
    return ll
def res_to_centroid(res):
    df=res_to_xyz(res)
    labels=df_to_labels(df)
    cent=get_centroid_ll(df,labels)
    return cent

In [19]:
def get_df(res):
    l=[]
    for r in res:
        try:
            l.append(lcd.loc[r])
        except Exception as e:
            #print(e,1)
            pass
    return pd.DataFrame(l)
def df_to_labels_2(df):
    kms_per_radian = 6371.0088
    eps = 8.0 / kms_per_radian
    db=DBSCAN(eps=eps,min_samples=2,algorithm='ball_tree', metric='haversine')
    db.fit_predict(np.radians(df))
    return db.labels_
def final_ll(df,labels):
    l=[]
    f={}
    for i in range(df.shape[0]):
        if labels[i]==-1:
            l.append(list(df.iloc[i]))
        else:
            lb=labels[i]
            if lb not in f:
                f[lb]=[]
            f[lb].append(list(df.iloc[i]))
    for lb in f:
        l.append(centroid(f[lb]))
    return l
def res_to_centroid_2(res):
    df=get_df(res)
    labels=df_to_labels_2(df)
    cent=final_ll(df,labels)
    return cent

In [20]:
def get_simplified(l):
    q=[]
    for i in l:
        for j in i:
            q.append(j)
    s=','.join([str(x) for x in q])
    return s

In [21]:
lcd=pd.read_csv('res_location.csv',index_col='res')
df1=pd.read_csv('user_del_res.csv')

In [22]:
l=[]
j=0
for i in range(df1.shape[0]):
    try:
        f={}
        f['user_id']=df1.iloc[i]['user_id']
        res=df1.iloc[i]['res_list'].split(',')
        f['centroid']=get_simplified(res_to_centroid_2(res))
        l.append(f)
        j+=1
        print(j,end='\r')
    except Exception as e:
        #print(e,2)
        pass
    #break
cent=pd.DataFrame(l)

In [23]:
cent.to_csv('user_del_cent.csv',index=False)

In [24]:
lcd=pd.read_csv('res_location.csv',index_col='res')
df1=pd.read_csv('user_res_list.csv')

In [25]:
l=[]
j=0
for i in range(df1.shape[0]):
    try:
        f={}
        f['user_id']=df1.iloc[i]['user_id']
        res=df1.iloc[i]['res_list'].split(',')
        f['centroid']=get_simplified(res_to_centroid_2(res))
        l.append(f)
        j+=1
        print(j,end='\r')
    except Exception as e:
        #print(e,2)
        pass
    #break
cent=pd.DataFrame(l)

In [26]:
cent.to_csv('user_res_cent.csv',index=False)